In [1]:
#Import Modules
import requests
import pandas as pd
import numpy as np

In [2]:
#Specify API end point URL
url = 'https://www.peeringdb.com/api/net'
#Use the requests package to make a GET request from the API endpoint:
r = requests.get(url)

In [3]:
#Print the status of our request
print(r.status_code)

200


In [4]:
json = r.json()
#look at the json keys, and then we’ll create our dataframe(s)
json.keys()

dict_keys(['data', 'meta'])

In [5]:
json_data = json['data']

In [6]:
#Create Dataframes from the data key we need
data_df = pd.DataFrame(json_data)

In [7]:
data_df.head()

,id,org_id,name,aka,name_long,website,asn,looking_glass,route_server,irr_as_set,...,poc_updated,policy_url,policy_general,policy_locations,policy_ratio,policy_contracts,allow_ixp_update,created,updated,status
0,1,8897,GTT Communications (AS4436),Formerly known as nLayer Communications,,http://www.gtt.net,4436,,,,...,2016-03-14T21:53:18Z,http://www.gtt.net/peering/,Restrictive,Required - International,True,Required,False,2004-07-28T00:00:00Z,2018-08-29T14:21:57Z,ok
1,2,14,Akamai Technologies,,,https://www.akamai.com/,20940,,,AS-AKAMAI,...,2021-05-11T21:26:32.758929Z,,Open,Not Required,False,Not Required,False,2004-07-28T00:00:00Z,2021-08-18T12:21:55Z,ok
2,3,17,DALnet IRC Network,,,http://www.dal.net,31800,,,AS31800,...,2016-03-14T21:22:01Z,http://peering.dal.net,Open,Preferred,False,Not Required,False,2004-07-28T00:00:00Z,2016-03-14T21:33:04Z,ok
3,4,18,Limelight Networks Global,llnw.net,,https://www.limelight.com,22822,,,AS-LLNW,...,2021-06-15T10:12:20.394183Z,https://www.limelight.com/peering/,Selective,Required - US,False,Not Required,True,2004-07-28T00:00:00Z,2020-07-01T14:21:57Z,ok
4,5,9350,Swisscom,IP-Plus,,http://www.swisscom.com,3303,,telnet://route-server.ip-plus.net,AS-SWCMGLOBAL,...,2020-01-22T04:24:08Z,https://extranet.swisscom.ch/ipplus/doc/Swissc...,Selective,Preferred,True,Required,False,2004-07-28T00:00:00Z,2021-10-07T08:37:54Z,ok


In [8]:
#Print out column names
data_df.columns

Index(['id', 'org_id', 'name', 'aka', 'name_long', 'website', 'asn',
       'looking_glass', 'route_server', 'irr_as_set', 'info_type',
       'info_prefixes4', 'info_prefixes6', 'info_traffic', 'info_ratio',
       'info_scope', 'info_unicast', 'info_multicast', 'info_ipv6',
       'info_never_via_route_servers', 'ix_count', 'fac_count', 'notes',
       'netixlan_updated', 'netfac_updated', 'poc_updated', 'policy_url',
       'policy_general', 'policy_locations', 'policy_ratio',
       'policy_contracts', 'allow_ixp_update', 'created', 'updated', 'status'],
      dtype='object')

In [9]:
#Select the columns we need
asn_df = data_df[['id', 'org_id', 'name','asn']]

In [10]:
asn_df.head()

,id,org_id,name,asn
0,1,8897,GTT Communications (AS4436),4436
1,2,14,Akamai Technologies,20940
2,3,17,DALnet IRC Network,31800
3,4,18,Limelight Networks Global,22822
4,5,9350,Swisscom,3303


In [11]:
asn_df.shape

(23214, 4)

In [12]:
#Checking % of missing data per column
asn_df.isnull().mean()*100

id        0.0
org_id    0.0
name      0.0
asn       0.0
dtype: float64

In [13]:
#Confirming that all asn numbers are unique with no duplicates
asn_df['asn'].unique().shape

(23214,)

We want to import all csv files in our import folder and compare the ASNs with the ASN from the endpoint
and include the organization name for all files in the directory.

In [16]:
import glob
import os
#Specifiy the directory with documents to be imported
#All
path = r"/Users/seyi/MyDocuments/ASN_Import/*.csv"
#Specifiy the directory with documents to be imported
directory = r"/Users/seyi/MyDocuments/ASN_Import"
iter_count = 0
#Empty list to store file names
doc_names = []
files = os.listdir(directory)
#Iterate over the files in the import directory and store their names in the doc_names list
for f in files:
    if f != '.DS_Store':
        doc_names.append(f)
for fname in glob.glob(path):
    # iterate over the files in the Import directory
    nig_df = pd.read_csv(fname, sep=';')
    #Corresponding organisation name for each ASN number is gotten using the map function and stored in a new column
    nig_df['Organization 1'] = nig_df['AS Number 1'].map(asn_df.set_index('asn').name)
    nig_df['Organization 2'] = nig_df['AS Number 2'].map(asn_df.set_index('asn').name)
    nig_df = nig_df[['AS Number 1', 'Organization 1', 'AS Number 2', 'Organization 2', 'Avg Total Throughput (bits/s)','Avg 1 to 2 Throughput (bits/s)', 'Avg 2 to 1 Throughput (bits/s)','Max Total Throughput (bits/s)', 'Max 1 to 2 Throughput (bits/s)','Max 2 to 1 Throughput (bits/s)', 'Connected Time (seconds)']]
    #this will replace "0" with "Organization Name"
    nig_df['Organization 1'] = np.where(nig_df['AS Number 1'] == 0, 'Main Telecoms Coy', nig_df['Organization 1'])
    nig_df['Organization 2'] = np.where(nig_df['AS Number 2'] == 0, 'Main Telecoms Coy', nig_df['Organization 2'])
    #Export Directory
    exp_dir ='/Users/seyi/MyDocuments/ASN_Export/Export_'
    file_name = exp_dir+str(doc_names[iter_count])+'.xlsx'
    #Export files to excel
    nig_df.to_excel(file_name, index=False)
    iter_count +=1
print(iter_count)

20
